#### **Transfer Learning**

In [86]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [87]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [88]:
from torchvision.transforms import transforms
from torchvision.models import vgg16

In [89]:
input_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [90]:
from PIL import Image
import numpy as np

In [91]:
class CustomDataset(Dataset):
    def __init__(self, features, labels, transform):
        self.features = features
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.features) 
    
    def __getitem__(self, index):
        # Resize to (28,28)
        image_path = self.features[index]
        
        image = Image.open(image_path).convert('RGB')
        
        # Change datatype to uint8
        # image = image.astype(np.uint8)
        
        # Change b&w to color, 1 -> 3 channels
        # image = np.stack([image]*3, axis=-1)
        
        # Convert array to PIL image
        # image = Image.fromarray(image)
        
        # Apply transformation
        image = self.transform(image)
        
        return image, torch.tensor(self.labels[index], dtype=torch.float)

In [92]:
from torchvision import datasets

In [93]:
train_dataset = datasets.ImageFolder(root='data/cat_vs_dog/training_set')
test_dataset = datasets.ImageFolder(root='data/cat_vs_dog/test_set')

In [94]:
train_features = [path for path, _ in train_dataset.samples]
train_labels = [label for _, label in train_dataset.samples]

test_features = [path for path, _ in test_dataset.samples]
test_labels = [label for _, label in test_dataset.samples]

In [95]:
train_dataset = CustomDataset(train_features, train_labels, input_transform)
test_dataset = CustomDataset(test_features, test_labels, input_transform)

In [96]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [97]:
model = vgg16(pretrained=True)

c:\Users\Ojas\.vscode\Repos\Deep-Learning\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Ojas\.vscode\Repos\Deep-Learning\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [98]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [99]:
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
12.1
1
NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [100]:
learning_rate = 0.0001
epochs = 10

In [101]:
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images).squeeze(1)
            probs = torch.sigmoid(outputs)
            predicted = (probs > 0.5).float()
            
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            total += labels.size(0)
            correct += (predicted.squeeze() == labels).sum().item()
    
    avg_loss = val_loss / len(data_loader)
    accuracy = 100 * correct / total
    
    return avg_loss, accuracy            

In [102]:
def train_model(model, train_loader, criterion, optimizer, device, epochs, val_loader=None, validation=False):
    history = {
        "training_loss": [],
        "validation_loss": [],
        "validation_accuracy": []
    }
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).float()
            optimizer.zero_grad()
            output = model(images).squeeze(1)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        epoch_loss = running_loss / len(train_loader)    
        history['training_loss'].append(epoch_loss)        
        
        if validation is True:
            val_loss, val_acc = evaluate_model(model, val_loader, criterion, device)
        
            # save history
            history['validation_loss'].append(val_loss)
            history['validation_accuracy'].append(val_acc)
            
            print(f"Epoch [{epoch+1}/{epochs}] "
                f"Train Loss: {epoch_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | "
                f"Val Acc: {val_acc:.2f}%")
        
    return  history         

In [103]:
for param in model.features.parameters():
    param.requires_grad = False

In [104]:
model.classifier = nn.Sequential(
    nn.Linear(25088,128),
    nn.ReLU(),
    nn.Dropout(0.5),
    
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Dropout(0.3),
    
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Dropout(0.2),
    
    nn.Linear(32,1)
)

In [105]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=learning_rate)

In [106]:
model = model.to(device)

In [107]:
history = train_model(model, train_loader, criterion, optimizer, device, epochs)

In [111]:
history

{'training_loss': [0.16132791357419168,
  0.03451355179053809,
  0.021365954313404947,
  0.012945852592409834,
  0.006760277770794404,
  0.004584479250380931,
  0.0030352861725755267,
  0.0015535389791581106,
  0.003178097814707002,
  0.0009841509579663727],
 'validation_loss': [],
 'validation_accuracy': []}

In [112]:
loss, accuracy = evaluate_model(model, test_loader, criterion, device)

In [113]:
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

Loss: 0.07526436547935589
Accuracy: 98.86307464162135
